# Basketball YOLOv8 using DirectML
Dataset obtained in:

https://universe.roboflow.com/roboflow-universe-projects/basketball-players-fy4c2/dataset/25

Version YOLOv8, not modified:

95% Train set (1140 img)

3% Valid set (32 img)

2% Train set (24 img)

## Librerías iniciales

In [ ]:
#Ultralytics update
#!pip install -U ultralytics

In [1]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import cv2
from ultralytics import YOLO
from IPython.display import Video
import numpy as np  
import seaborn as sns
sns.set(style='darkgrid')
import pathlib
import glob
from tqdm.notebook import trange, tqdm
import warnings
warnings.filterwarnings('ignore')

In [2]:
#To get current PATH
#os.getcwd()

## Plotting random train samples

In [ ]:
#Image directory for train samples
Image_dir = '/mnt/c/Users/alide/IA/Proyecto/BasketballYv8/train/images'

#To show samples
num_samples = 9
image_files = os.listdir(Image_dir)

# Randomly select num_samples images for plotting
rand_images = random.sample(image_files, num_samples)

fig, axes = plt.subplots(3, 3, figsize=(11, 11))

for i in range(num_samples):
    image = rand_images[i]
    ax = axes[i // 3, i % 3]
    ax.imshow(plt.imread(os.path.join(Image_dir, image)))
    ax.set_title(f'Image {i+1}')
    ax.axis('off')

plt.tight_layout()
plt.show()

## YOLOv8 Training

In [ ]:
# Use a pretrained YOLOv8n model
model = YOLO("yolov8n.pt") 

In [ ]:
# Use the model to detect object
image = "/mnt/c/Users/alide/IA/Proyecto/BasketballYv8/train/images/youtube-1_jpg.rf.771c14fa7ff8ea51f5f1be48cba8e331.jpg"
result_predict = model.predict(source = image, imgsz=(640))

# show results
plot = result_predict[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

In [ ]:
# Build from YAML and transfer weights
Training_model = YOLO('yolov8n.pt')

In [ ]:
#Training
Result_Training_model = Training_model.train(data="/mnt/c/Users/alide/IA/Proyecto/BasketballYv8/data.yaml", device=0
                                             epochs = 1, batch = 4, optimizer = 'auto')

## Post-training Results

In [ ]:
#Function for showing results
def display_images(post_training_files_path, image_files):

    for image_file in image_files:
        image_path = os.path.join(post_training_files_path, image_file)
        img = cv2.imread(image_path)
        if img is None:
            print(f"❌ Could not read image: {image_path}")
            continue
        
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        plt.figure(figsize=(10, 10), dpi=120)
        plt.imshow(img)
        plt.axis('off')
        plt.show()

# List of image files to display
image_files = [
    'confusion_matrix_normalized.png',
    'F1_curve.png',
    'P_curve.png',
    'R_curve.png',
    'PR_curve.png',
    'results.png'
]

# Path to the directory containing the images
post_training_files_path = '/home/alide/Notebooks/ProyectoPC/runs/detect/train'

# Display the images
display_images(post_training_files_path, image_files)

## Best performing model and example

In [ ]:
# Loading the best performing model
Final_model = YOLO('/home/alide/Notebooks/ProyectoPC/runs/detect/train/weights/best.pt')

# Evaluating the model on the validset
metrics = Final_model.val(split = 'val')

# final results 
print("precision(B): ", metrics.results_dict["metrics/precision(B)"])
print("metrics/recall(B): ", metrics.results_dict["metrics/recall(B)"])
print("metrics/mAP50(B): ", metrics.results_dict["metrics/mAP50(B)"])
print("metrics/mAP50-95(B): ", metrics.results_dict["metrics/mAP50-95(B)"])

### Exporting the model

In [ ]:
#Install dependencies
!pip install onnx
!pip install onnxruntime

In [ ]:
#Export the model
Final_model.export(format='onnx')

### Example using Final_Model with the same img used with the pre trained model

In [ ]:
image = "/mnt/c/Users/alide/IA/Proyecto/BasketballYv8/train/images/youtube-1_jpg.rf.771c14fa7ff8ea51f5f1be48cba8e331.jpg"
result_predict = Final_model.predict(source = image, imgsz=(640))

# show results
plot = result_predict[0].plot()
plot = cv2.cvtColor(plot, cv2.COLOR_BGR2RGB)
display(Image.fromarray(plot))

## FInal Model architecture

In [ ]:
Final_model